In [1]:
%load_ext Cython

In [2]:
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

In [3]:
import os
from typing import Tuple,Callable,Dict,Optional,List

import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import random
from scipy.sparse import csr_matrix
import implicit 

from sklearn.model_selection import train_test_split

**Dataset loading with pandas**

The function read_csv from pandas provides a wonderful and fast interface to load tabular data like this. For better results and performance we provide the separator ::, the column names ["user_id", "item_id", "ratings", "timestamp"], and the types of each attribute in the dtype parameter.

In [4]:
def load_data():
  return pd.read_csv("./data_train.csv")

In [5]:
ratings=load_data()
d ={'user_id': ratings['row'],'item_id':ratings['col'],'ratings':ratings['data']}
ratings=pd.DataFrame(data=d)

In [6]:
ratings.dtypes

user_id      int64
item_id      int64
ratings    float64
dtype: object

In [7]:
userList=list(d['user_id'])
itemList=list(d['item_id'])
ratingList=list(d['ratings'])

In [8]:
URM_all = sp.coo_matrix((ratingList,(userList,itemList)))
URM_all = URM_all.tocsr()

In [9]:
URM_all

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 113268 stored elements in Compressed Sparse Row format>

In [10]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [11]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [12]:
featureList=list(d['feature_id'])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(featureList)

featureList = le.transform(featureList)

In [13]:
valueList=list(d['value'])
ICM_all = sp.coo_matrix((valueList,(itemList,featureList)))
ICM_all = ICM_all.tocsr()

In [14]:
ICM_all

<25975x19998 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

This section wors with the previously-loaded ratings dataset and extracts the number of users, number of items, and min/max user/item identifiers. Exploring and understanding the data is an essential step prior fitting any recommender/algorithm.

In this specific case, we discover that item identifiers go between 1 and 25974, however, there are only 24896 different items. To ease further calculations, we create new contiguous user/item identifiers, we then assign each user/item only one of these new identifiers. To keep track of these new mappings, we add them into the original dataframe using the pd.merge function.

In [15]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()
    
    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()
    
    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)
    
    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})
    
    ratings = pd.merge(left=ratings, 
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")
    
    ratings = pd.merge(left=ratings, 
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")
    
    return ratings


In [16]:
ratings_mapped=preprocess_data(ratings)

7947 0 7946
24896 0 25974


In [17]:
ratings_mapped

,user_id,item_id,ratings,mapped_user_id,mapped_item_id
0,0,10080,1.0,0,0
1,4342,10080,1.0,4342,0
2,5526,10080,1.0,5526,0
3,5923,10080,1.0,5923,0
4,0,19467,1.0,0,1
5,149,19467,1.0,149,1
6,4072,19467,1.0,4072,1
7,6193,19467,1.0,6193,1
8,7105,19467,1.0,7105,1
9,1,2665,1.0,1,2


In [18]:
unique_users = ratings_mapped.mapped_user_id.unique()
unique_items = ratings_mapped.mapped_item_id.unique()
num_users=unique_users.size
num_items=unique_items.size

In [19]:
num_users

7947

In [20]:
num_items

24896

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [21]:
def dataset_splits(ratings, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234
    
    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     ratings_training, ratings_test) = train_test_split(ratings.mapped_user_id,
                                                        ratings.mapped_item_id,
                                                        ratings.ratings,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)
    
    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     ratings_training, ratings_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              ratings_training,
                                                              test_size=validation_percentage,
                                                             )
    
    urm_train = sp.csr_matrix((ratings_training, (user_ids_training, item_ids_training)), 
                              shape=(num_users, num_items))
    
    
    urm_validation = sp.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)), 
                              shape=(num_users, num_items))
    
    urm_test = sp.csr_matrix((ratings_test, (user_ids_test, item_ids_test)), 
                              shape=(num_users, num_items))
    

    
    return urm_train, urm_validation, urm_test



In [22]:
urm_train, urm_validation, urm_test = dataset_splits(ratings_mapped, 
                                                     num_users, 
                                                     num_items, 
                                                     validation_percentage=0.10, 
                                                     testing_percentage=0.15)

In [23]:
urm_train_validation = urm_train + urm_validation

**Evaluation Metrics**

In this practice session we will be using the same evaluation metrics defined in the Practice session 2, i.e., precision, recall and mean average precision (MAP).

In [24]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [25]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]
        
        relevant_items = urm_test.indices[user_profile_start:user_profile_end]
        
        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
        recommendations = recommender.recommend(user_id=user_id,
                                               urm_train = urm_train,
                                               at=recommendation_length
                                               #remove_seen=True)
                                               )
        
        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)
        
        num_users_evaluated += 1
        
    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

**List of recommenders**

**1)ItemBasedCF**

In [26]:
class ItemBasedCollaborativeFiltering(object):
    def __init__(self, knn = 100, shrink = 5, similarity="cosine"):
        self.knn = knn
        self.shrink = shrink
        self.similarity = similarity
        self.URM = None
        self.SM_item = None

    def generate_similarity_matrix(self):
        similarity_object = Compute_Similarity_Python(self.URM, topK =self.knn, shrink = self.shrink, normalize=True, similarity=self.similarity)
        return similarity_object.compute_similarity()

    def fit(self, URM):
        self.URM = URM
        self.URM = self.URM.tocsr()
        self.SM_item = self.generate_similarity_matrix()
        self.RECS = self.URM.dot(self.SM_item)

    def get_expected_ratings(self,user_id):
        expected_ratings = self.RECS[user_id].todense()
        return np.squeeze(np.asarray(expected_ratings))

    def recommend(self, user_id,urm_train: sp.csr_matrix, at=10):
        expected_ratings = self.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0) #Ordino gli expected ratings

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)
        recommended_items = recommended_items[unseen_items_mask]
        return recommended_items[0:at]


In [27]:
itemCF=ItemBasedCollaborativeFiltering(knn = 100, shrink = 5, similarity="cosine")
itemCF.fit(urm_train)

Similarity column 24896 ( 100 % ), 2613.55 column/sec, elapsed time 0.16 min


In [28]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

In [29]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.02208576605596919, 0.0822057498887981, 0.03799173537537208, 4967, 2980)

**2)UserBasedCF**

In [30]:
class UserBasedCollaborativeFiltering(object):
    def __init__(self, knn=300, shrink=4, similarity="cosine"):
        self.knn = knn
        self.shrink = shrink
        self.similarity = similarity
        self.URM = None
        self.SM_user = None

    def generate_similarity_matrix(self):
        similarity_object = Compute_Similarity_Python(self.URM.transpose(), self.knn, self.shrink, True,
                                                      self.similarity)
        return similarity_object.compute_similarity()

    def fit(self, URM):
        self.URM = URM
        self.SM_user = self.generate_similarity_matrix()
        self.RECS = self.SM_user.dot(self.URM)

    def get_expected_ratings(self,user_id):
        expected_ratings = self.RECS[user_id].todense()
        return np.squeeze(np.asarray(expected_ratings))

    def recommend(self,user_id, urm_train: sp.csr_matrix,at=10):
        expected_ratings = self.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices, assume_unique=True, invert=True)
        recommended_items = recommended_items[unseen_items_mask]
        return recommended_items[0:at]

In [31]:
userCF=UserBasedCollaborativeFiltering(knn = 100, shrink = 5, similarity="cosine")
userCF.fit(urm_train)

Similarity column 7947 ( 100 % ), 3822.53 column/sec, elapsed time 0.03 min


In [32]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

In [33]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.028749748339037193, 0.1074955744504704, 0.05116012422350322, 4967, 2980)

**3) ALS**

In [34]:
class AlternatingLeastSquare:

    def __init__(self, n_factors, regularization, iterations): #Dobbiamo passarglieli come parametri
        self.n_factors = n_factors
        self.regularization = regularization
        self.iterations = iterations

    def fit(self, URM):
        self.URM = URM

        sparse_item_user = self.URM.T

        # Initialize the als model and fit it using the sparse item-user matrix
        model = implicit.als.AlternatingLeastSquares(factors=self.n_factors, regularization=self.regularization, iterations=self.iterations)


        alpha_val = 24
        # Calculate the confidence by multiplying it by our alpha value.

        data_conf = (sparse_item_user * alpha_val).astype('double')

        # Fit the model
        model.fit(data_conf)

        # Get the user and item vectors from our trained model
        self.user_factors = model.user_factors
        self.item_factors = model.item_factors


    def get_expected_ratings(self, user_id):
        scores = np.dot(self.user_factors[user_id], self.item_factors.T)

        return np.squeeze(scores)

    def recommend(self, user_id, urm_train: sp.csr_matrix, at=10):
        
        expected_ratings = self.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items, urm_train[user_id].indices,
                                    assume_unique=True, invert=True)
        recommended_items = recommended_items[unseen_items_mask]
        return recommended_items[0:at]

In [35]:
ALS=AlternatingLeastSquare(n_factors=300, regularization=0.15, iterations=30)
ALS.fit(urm_train)

In [36]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

In [37]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.02903160861687089, 0.11126360952405162, 0.04891599335787518, 4967, 2980)

**CBF**

In [ ]:
#Mia versione di preprocess
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
URM_train_validation_all = URM_train + URM_validation

In [38]:
class ContentBasedFiltering(object):

    def __init__(self, knn=100, shrink=2):
        self.knn = knn
        self.shrink = shrink

    def compute_similarity(self, ICM, knn, shrink):
        similarity_object = Compute_Similarity_Python(ICM.transpose(), topK=knn, shrink=shrink,
                                                     normalize=True, similarity="cosine")
        return similarity_object.compute_similarity()

    def fit(self, URM, ICM):

        self.URM = URM
        self.ICM = ICM

        self.SM = self.compute_similarity(self.ICM, self.knn, self.shrink)

    def get_expected_ratings(self, user_id):
        
        items_all = self.URM[user_id] #URM_all
        expected_ratings_all_items = items_all.dot(self.SM).toarray().ravel()
        
        return expected_ratings_all_items

    def recommend(self, user_id, urm_train: sp.csr_matrix, at):
        
        expected_ratings_all_items = self.get_expected_ratings(user_id)

        recommended_items = np.flip(np.argsort(expected_ratings_all_items), 0)
        return recommended_items[0:at]


In [39]:
CBF_all=ContentBasedFiltering(knn=50, shrink=100)
CBF_all.fit(URM_all, ICM_all)

Similarity column 25975 ( 100 % ), 1259.20 column/sec, elapsed time 0.34 min


In [40]:
#Se si dà urm minuscola valori bassissimi, se si dà URM maiuscola (splittata con l'altra versione) viene un risultato possibile
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(CBF_all, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

In [41]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00014093013891685122,
 0.00040390234497428946,
 0.0002448301648019788,
 4967,
 2980)

**Hybrid**

In [ ]:
"""def itemFeatureNoRatings() :
    
    founded = False
    
    items=[]
    
    for item_id_all in range(0,URM_all.shape[1]) :
        
        founded = False
        
        for item_id_ratings in ratings.item_id.unique() :
            
            if item_id_all == item_id_ratings :
                founded = True
        
        if founded == False : 
            items.append(item_id_all)
    
    return items
    """

In [42]:
itemListNoRatings = [7, 38, 50, 124, 144, 174, 243, 266, 270, 277, 288, 320, 347, 464, 470, 494, 501, 542, 565, 573, 629, 651, 665, 698, 760, 769, 887, 949, 966, 1012, 1052, 1059, 1117, 1156, 1210, 1227, 1234, 1263, 1280, 1298, 1315, 1319, 1363, 1402, 1407, 1481, 1501, 1519, 1527, 1536, 1548, 1556, 1562, 1565, 1568, 1570, 1582, 1605, 1661, 1675, 1680, 1746, 1774, 1812, 1839, 1881, 1922, 1966, 2010, 2032, 2063, 2098, 2107, 2123, 2162, 2208, 2212, 2221, 2224, 2257, 2262, 2264, 2301, 2319, 2335, 2337, 2367, 2391, 2417, 2425, 2435, 2506, 2514, 2524, 2544, 2574, 2603, 2625, 2680, 2686, 2695, 2699, 2735, 2738, 2814, 2828, 2883, 2891, 2984, 3006, 3007, 3022, 3023, 3043, 3069, 3072, 3088, 3090, 3091, 3095, 3096, 3110, 3126, 3159, 3171, 3179, 3204, 3240, 3259, 3284, 3307, 3311, 3316, 3330, 3335, 3347, 3353, 3375, 3406, 3411, 3412, 3419, 3434, 3454, 3459, 3476, 3487, 3489, 3638, 3704, 3723, 3786, 3791, 3798, 3824, 3847, 3854, 3887, 3911, 3914, 3963, 3993, 4058, 4109, 4132, 4141, 4214, 4219, 4227, 4244, 4255, 4278, 4303, 4327, 4378, 4405, 4419, 4450, 4462, 4467, 4468, 4484, 4496, 4506, 4540, 4573, 4600, 4609, 4610, 4613, 4617, 4669, 4717, 4734, 4776, 4798, 4808, 4836, 4840, 4924, 4931, 4949, 4962, 4972, 4999, 5014, 5019, 5023, 5040, 5056, 5060, 5075, 5089, 5104, 5105, 5109, 5120, 5134, 5162, 5175, 5184, 5197, 5233, 5237, 5239, 5240, 5264, 5266, 5271, 5323, 5375, 5398, 5441, 5483, 5511, 5531, 5536, 5544, 5553, 5580, 5590, 5638, 5674, 5691, 5747, 5755, 5775, 5779, 5814, 5839, 5842, 5849, 5881, 5927, 5935, 6008, 6009, 6020, 6025, 6032, 6143, 6154, 6174, 6190, 6214, 6216, 6223, 6258, 6305, 6328, 6347, 6370, 6387, 6411, 6507, 6571, 6645, 6678, 6694, 6724, 6750, 6779, 6798, 6804, 6873, 6896, 6938, 6941, 6984, 6999, 7040, 7052, 7071, 7101, 7161, 7209, 7231, 7252, 7256, 7276, 7287, 7297, 7303, 7305, 7342, 7348, 7376, 7382, 7403, 7406, 7411, 7425, 7490, 7515, 7523, 7576, 7592, 7660, 7685, 7757, 7762, 7772, 7774, 7786, 7811, 7848, 7854, 7862, 7881, 7904, 7947, 7949, 7952, 7953, 7976, 8001, 8022, 8034, 8039, 8073, 8076, 8111, 8119, 8125, 8156, 8166, 8177, 8178, 8202, 8225, 8281, 8297, 8307, 8328, 8366, 8368, 8398, 8404, 8434, 8443, 8453, 8455, 8460, 8466, 8476, 8501, 8548, 8592, 8632, 8646, 8710, 8719, 8774, 8777, 8801, 8827, 8885, 8890, 8904, 8911, 8915, 8923, 8978, 8989, 9013, 9025, 9031, 9032, 9048, 9108, 9137, 9148, 9153, 9161, 9169, 9180, 9198, 9235, 9237, 9254, 9287, 9376, 9434, 9453, 9467, 9481, 9517, 9518, 9568, 9599, 9624, 9626, 9638, 9656, 9671, 9674, 9677, 9758, 9827, 9878, 9881, 9906, 9936, 9964, 10008, 10053, 10088, 10154, 10192, 10250, 10253, 10267, 10333, 10354, 10361, 10402, 10435, 10442, 10474, 10486, 10487, 10492, 10533, 10548, 10553, 10562, 10571, 10631, 10635, 10650, 10705, 10819, 10828, 10854, 10858, 10861, 10909, 10925, 10943, 10947, 10978, 11015, 11016, 11017, 11026, 11058, 11070, 11077, 11105, 11113, 11123, 11136, 11140, 11155, 11165, 11249, 11308, 11309, 11330, 11368, 11374, 11391, 11412, 11415, 11454, 11488, 11526, 11544, 11551, 11560, 11580, 11598, 11601, 11621, 11629, 11632, 11648, 11688, 11751, 11768, 11807, 11828, 11843, 11891, 11920, 11927, 11932, 11973, 11977, 11990, 12012, 12015, 12064, 12071, 12073, 12075, 12084, 12095, 12112, 12115, 12144, 12150, 12155, 12233, 12297, 12303, 12331, 12370, 12387, 12412, 12416, 12434, 12472, 12477, 12485, 12502, 12516, 12529, 12539, 12559, 12623, 12635, 12696, 12765, 12766, 12771, 12784, 12797, 12814, 12822, 12866, 12871, 12880, 12896, 12930, 12958, 12984, 13033, 13050, 13085, 13092, 13209, 13212, 13264, 13273, 13285, 13292, 13364, 13365, 13496, 13515, 13551, 13557, 13568, 13572, 13580, 13584, 13623, 13647, 13666, 13669, 13680, 13687, 13694, 13741, 13754, 13790, 13844, 13857, 13924, 13992, 13997, 14004, 14040, 14046, 14055, 14071, 14073, 14103, 14107, 14111, 14187, 14228, 14239, 14246, 14267, 14284, 14297, 14349, 14353, 14391, 14594, 14611, 14629, 14664, 14710, 14712, 14729, 14754, 14768, 14781, 14786, 14791, 14803, 14840, 14901, 14916, 14956, 14968, 15000, 15037, 15075, 15096, 15111, 15127, 15134, 15168, 15170, 15250, 15252, 15267, 15281, 15301, 15305, 15306, 15308, 15340, 15348, 15360, 15389, 15393, 15435, 15442, 15458, 15472, 15492, 15555, 15558, 15600, 15606, 15619, 15620, 15653, 15690, 15726, 15735, 15747, 15757, 15788, 15819, 15820, 15848, 15851, 15858, 15870, 15879, 15885, 15889, 15921, 15931, 15933, 15962, 15970, 15974, 15978, 16008, 16028, 16033, 16037, 16041, 16060, 16102, 16113, 16138, 16159, 16210, 16256, 16292, 16296, 16306, 16329, 16332, 16337, 16344, 16352, 16386, 16470, 16474, 16479, 16483, 16507, 16512, 16540, 16557, 16565, 16573, 16586, 16638, 16673, 16750, 16759, 16768, 16797, 16831, 16863, 16930, 16959, 16962, 16963, 16988, 16994, 17002, 17011, 17035, 17098, 17124, 17139, 17154, 17182, 17222, 17272, 17281, 17307, 17354, 17408, 17449, 17477, 17488, 17510, 17555, 17582, 17597, 17627, 17667, 17673, 17676, 17687, 17703, 17706, 17751, 17792, 17817, 17831, 17859, 17862, 17869, 17871, 17881, 17980, 18019, 18040, 18113, 18138, 18162, 18198, 18203, 18221, 18225, 18280, 18386, 18409, 18492, 18519, 18582, 18590, 18613, 18618, 18629, 18658, 18660, 18663, 18678, 18683, 18734, 18763, 18833, 18840, 18842, 18850, 18875, 18888, 18897, 18920, 18933, 18967, 19024, 19031, 19143, 19172, 19181, 19183, 19214, 19222, 19234, 19264, 19265, 19266, 19268, 19272, 19314, 19334, 19349, 19383, 19391, 19435, 19440, 19449, 19523, 19556, 19559, 19567, 19583, 19636, 19688, 19703, 19741, 19763, 19787, 19791, 19860, 19902, 19936, 20020, 20040, 20124, 20126, 20185, 20198, 20246, 20303, 20338, 20346, 20364, 20407, 20440, 20463, 20503, 20504, 20527, 20535, 20559, 20567, 20575, 20580, 20653, 20682, 20687, 20721, 20723, 20741, 20753, 20783, 20886, 20903, 20996, 21006, 21032, 21045, 21063, 21065, 21118, 21126, 21148, 21157, 21202, 21214, 21215, 21231, 21243, 21279, 21280, 21290, 21348, 21361, 21412, 21424, 21426, 21427, 21460, 21485, 21487, 21546, 21555, 21562, 21569, 21638, 21690, 21718, 21736, 21753, 21782, 21788, 21821, 21832, 21833, 21884, 21915, 21959, 21964, 21983, 22029, 22126, 22149, 22160, 22167, 22173, 22179, 22200, 22207, 22228, 22273, 22279, 22289, 22304, 22323, 22367, 22368, 22416, 22427, 22432, 22471, 22520, 22541, 22612, 22620, 22624, 22663, 22677, 22683, 22698, 22711, 22716, 22733, 22737, 22761, 22773, 22776, 22782, 22783, 22832, 22944, 22988, 22992, 23029, 23058, 23091, 23097, 23131, 23139, 23180, 23191, 23201, 23209, 23229, 23250, 23293, 23303, 23326, 23364, 23413, 23479, 23484, 23498, 23558, 23567, 23574, 23601, 23611, 23617, 23639, 23681, 23716, 23740, 23761, 23765, 23786, 23843, 23874, 23935, 23948, 23964, 23998, 24000, 24004, 24023, 24026, 24028, 24034, 24035, 24054, 24058, 24064, 24103, 24119, 24131, 24141, 24155, 24180, 24273, 24302, 24332, 24353, 24370, 24409, 24430, 24435, 24486, 24490, 24492, 24507, 24516, 24571, 24640, 24698, 24729, 24755, 24759, 24772, 24779, 24814, 24820, 24826, 24931, 24940, 24980, 24982, 24983, 24986, 25057, 25072, 25090, 25103, 25132, 25140, 25228, 25246, 25264, 25274, 25308, 25323, 25431, 25447, 25455, 25469, 25527, 25535, 25551, 25588, 25601, 25671, 25684, 25686, 25691, 25729, 25899, 25904, 25907, 25936, 25949]

In [43]:
#Funzione che serve a controllare se un item_id è nelle features ma non nei ratings
def isFeaturesNoRatings(item_id : int, itemListNoRatings) :
        
    if item_id not in itemListNoRatings :
        return False
    
    return True
    

In [44]:
result = isFeaturesNoRatings(2891, itemListNoRatings)
result

NameError: name 'ciao' is not defined

In [45]:
class HybridRecommender(object):
    def __init__(self, w, user_cf_param, item_cf_param):

        self.w=w
        ###### USER CF #####
        self.userCF = UserBasedCollaborativeFiltering(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"])
    
        ###### ITEM_CF #####
        self.itemCF = ItemBasedCollaborativeFiltering(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"])
        
        ###### CBF ######
        self.CBF_all = ContentBasedFiltering(knn=cbf_param["knn"], shrink=cbf_param["shrink"])
    

    def fit(self, URM, URM_all, ICM):
        self.URM = URM
        self.URM_all = URM_all
        self.ICM = ICM

        ### SUB-FITTING ###
        print("Fitting user cf...")
        self.userCF.fit(URM.copy())

        print("Fitting item cf...")
        self.itemCF.fit(URM.copy())
        
        print("Fitting cbf...")
        self.CBF_all.fit(URM_all.copy(), ICM.copy())


    def recommend(self,user_id,urm_train: sp.csr_matrix,at=10):


        ### DUE TO TIME CONSTRAINT THE CODE STRUCTURE HERE IS REDUNTANT
        ### TODO exploit inheritance to reduce code duplications and simple extract ratings, combine them, simply by iterate over a list of recommenders
        self.userCF_ratings = self.userCF.get_expected_ratings(user_id)
        self.itemCF_ratings = self.itemCF.get_expected_ratings(user_id)
        
        self.CBF_all_ratings = self.CBF_all.get_expected_ratings(user_id) #Array con 25975 items

        ### COMMON CODE ### The array in which I'll sum all ratings excluded CBF, 24896 items
        self.hybrid_ratings = None #BE CAREFUL, MAGIC INSIDE :)

        self.hybrid_ratings = self.userCF_ratings * w["user_cf"]
        self.hybrid_ratings += self.itemCF_ratings * w["item_cf"]
        #self.hybrid_ratings += self.cbf_ratings * w_right["cbf"]
        #self.hybrid_ratings += self.slim_ratings * w_right["slim"]
        #self.hybrid_ratings += self.ALS_ratings * w_right["als"]
        #self.hybrid_ratings += self.slim_elastic_ratings * w_right["elastic"]
        
        #Applico il peso del CBF ai suoi ratings
        self.CBF_all_ratings = self.CBF_all_ratings * w["cbf"]
        
        #Ciclo for per sommare e nei punti in cui ci sono item nuovi blocco gli altri index
        counterItemInRatings = 0
        counter = 0
        
        print("Rating item 7 pre cicle: " + str(self.CBF_all_ratings[7]))
        print("Rating item 5 pre cicle: " + str(self.CBF_all_ratings[5]))
        print("Rating nell'array hybrid item 5: " + str(self.hybrid_ratings[5]))
        for counter in range(0, self.CBF_all_ratings.shape[0]) : #Ciclo fino a 25975
            
            if counter not in itemListNoRatings :
                
                self.CBF_all_ratings[counter] += self.hybrid_ratings[counterItemInRatings]

                counterItemInRatings = counterItemInRatings + 1
        
        
        recommended_items = np.flip(np.argsort(self.CBF_all_ratings), 0)
        print("Rating item 7 post cicle: " + str(self.CBF_all_ratings[7]))
        print("Rating item 5 post cicle: " + str(self.CBF_all_ratings[5]))
        print(str(user_id))

        # REMOVING SEEN
        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                    assume_unique=True, invert=True)
        recommended_items = recommended_items[unseen_items_mask]

        return recommended_items[0:at]

#IDEA: per il ContentBase 


################################ PARAMETERS #################################
#################### Value for best sumbissions, MAP@10 on Kaggle = 0.1 ###########################

w = {
    "user_cf": 0.5,
    "item_cf": 0.5,
    "cbf": 1
   # "icm_svd": 0,
   # "als": 0.3,
   # "slim": 0.6,
   # "elastic": 1.5
}

user_cf_param = {
    "knn": 140,
    "shrink": 100
}

item_cf_param = {
    "knn": 310,
    "shrink": 100
}

cbf_param = {
    "knn": 50,
    "shrink": 100
}



In [46]:
recommender = HybridRecommender(w=w,user_cf_param=user_cf_param,
                                item_cf_param=item_cf_param)

In [47]:
recommender.fit(urm_train, URM_all, ICM_all)

Fitting user cf...
Similarity column 7947 ( 100 % ), 4021.25 column/sec, elapsed time 0.03 min
Fitting item cf...
Similarity column 24896 ( 100 % ), 2724.03 column/sec, elapsed time 0.15 min
Fitting cbf...
Similarity column 25975 ( 100 % ), 1175.40 column/sec, elapsed time 0.37 min


In [48]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.27244303841143847
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.27244303841143847
1
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
2
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
3
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
4
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.03041743440553546
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.03041743440553546
5
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 

Rating nell'array hybrid item 5: 0.28640343714505434
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.3041444164700806
139
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.013396292924880981
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.013396292924880981
141
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 1.0329396934248507
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 1.0329396934248507
142
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.004448015708476305
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.004448015708476305
143
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
144
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.009029879234731197
Rating item 7 post cic

Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
309
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.008588727563619614
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.008588727563619614
311
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.010240599513053894
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.010240599513053894
312
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.07215598551556468
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.07215598551556468
315
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
316
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating it

Rating item 5 post cicle: 0.007142055779695511
451
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.08076298888772726
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.08076298888772726
452
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
453
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.06389994639903307
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.06389994639903307
455
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
456
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
458
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5:

Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
588
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.022029301151633263
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.022029301151633263
590
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.20660773431882262
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.20660773431882262
592
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
594
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.04904772201552987
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.04904772201552987
595
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item

...

Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7231
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7232
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7233
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7235
Rating item 7 pre cicle: 0.005223536863923073
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.005223536863923073
Rating item 5 post cicle: 0.0
7236
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.052433365024626255
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.052433365024626255
7238
R

Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7381
Rating item 7 pre cicle: 0.019040276762098074
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.2923333956860006
Rating item 7 post cicle: 0.019040276762098074
Rating item 5 post cicle: 0.2923333956860006
7382
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7384
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.011606278829276562
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.011606278829276562
7386
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7387
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7388


Rating item 5 post cicle: 0.0
7521
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7522
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7523
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7526
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.004728375468403101
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.004728375468403101
7527
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7529
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating

Rating item 5 post cicle: 0.0
7660
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7661
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7663
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7664
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7666
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.02325903531163931
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.02325903531163931
7668
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating i

Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7801
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.008492696098983288
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.008492696098983288
7802
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7803
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.0
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.0
7804
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.028621090576052666
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.028621090576052666
7805
Rating item 7 pre cicle: 0.0
Rating item 5 pre cicle: 0.0
Rating nell'array hybrid item 5: 0.007833391427993774
Rating item 7 post cicle: 0.0
Rating item 5 post cicle: 0.007833391427993774
7

In [49]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.001228105496275417,
 0.0034756564839301795,
 0.0013341306591759577,
 4967,
 2980)

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [ ]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [ ]:
goodguys

In [ ]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

In [ ]:
mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))

In [ ]:
mapping_to_item_id

In [ ]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
    items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()
    
    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))
    
    
    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id
        
        recommendations = recommender.recommend(user_id=mapped_user_id,
                                                urm_train=urm_train,
                                                at=recommendation_length,
                                                #remove_seen=True)
                                               )
        
        submission.append(user_id, [for item_id in recommendations])
        
    return submission

In [ ]:
submission = prepare_submission(ratings, users_to_recommend, urm_train_validation,recommender)

In [ ]:
submission

In [ ]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
write_submission(submission)